# My first project 

When creating a project, we start defining the infrastructure layout via two files:

  - ansible.cfg
  - inventory
  
Our current setup is:

  - direct access to client machines
  - one `inventory` file
  - no host_key_check
  - no retry files
  - client machine username/password is `root:root`
  - no pki certificate is installed by default



In [1]:
cd exercise-01/


/notebooks/exercise-01


## Preparation

The first steps include creating:

 - all secret files
 - the deployment descriptor 
 - and the inventory.

In [3]:
# At first create a proper ssh key for the project

! rm -f id_ansible && ssh-keygen -q -t ecdsa -f id_ansible  -N ''

# We'll reference this secret

In [4]:
# Now a `secret` password ;)
!echo secret > .pin    

In [5]:
# then prepare a deployment descriptor referencing the .pin file
!cat ansible.cfg


[defaults]
inventory = inventory
retry_files_enabled = false

#
# ssh configs
#
host_key_checking = no
private_key_file = id_ansible

# We want to store root password in an encrypted file
#  but don't want to be prompted for a password each time
vault_password_file = .pin


### Exercise

 - use ansible docs or the web to comment properly all the [ansible.cfg](/edit/notebooks/exercise-01/ansible.cfg) params

In [6]:
# And in the end, the inventory. 
!cat inventory

#
# This is the inventory file, containing all 
#  ansible hosts.
#
# - comments starts with #
# - variables are between {{ variable_name }}
#
[course]
172.17.0.[2:4]  ansible_password={{root_password}}  # load variable from vault



## Gotta ping 'em all

Everything set up now. 

Ping all hosts now!

In [7]:
# Let's get an error: root_password is UNDEFINED. 
!ansible -m ping all


172.17.0.2 | FAILED! => {
    "failed": true, 
    "msg": "the field 'password' has an invalid value, which appears to include a variable that is undefined. The error was: 'root_password' is undefined"
}
172.17.0.3 | FAILED! => {
    "failed": true, 
    "msg": "the field 'password' has an invalid value, which appears to include a variable that is undefined. The error was: 'root_password' is undefined"
}
172.17.0.4 | FAILED! => {
    "failed": true, 
    "msg": "the field 'password' has an invalid value, which appears to include a variable that is undefined. The error was: 'root_password' is undefined"
}


## Vaults

We now ensure that everything is encrypted and we're able to connect to some hosts

In [8]:
# A vault is just a yaml file containing a dictionary of secrets.
#  We can put here as many information as we want, but for now
#  just put the `root_password`.

!echo "root_password: root" > vaultfile.yml

In [9]:
# We need to encrypt it. 
!ansible-vault encrypt vaultfile.yml

Encryption successful


### Exercise

  - which secret is used to encrypt vaultfile.yml ?
  - where is it specified?

In [10]:
# And show the anatomy of the vault.
!cat vaultfile.yml

$ANSIBLE_VAULT;1.1;AES256
33666661333130643535313164353132306561333033643534323261306331383062336638333163
3464343638653630643865633862646664386663326534360a386165616437373964313663323962
66653131376435623837386264616261333330373932323532633966363334656566333766666364
3036396230336234340a653438343936663134653661316638653030373266653362643964396530
66393136396233623262613233613638313232633430373930316561323032306137


In [11]:
# Can we decrypt it?
!ansible-vault view vaultfile.yml

root_password: root


In [12]:
!ansible -m ping all -e@vaultfile.yml


172.17.0.2 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.17.0.2 port 22: Connection refused\r\n", 
    "unreachable": true
}
172.17.0.4 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
172.17.0.3 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


## Exercise

Can you pass the vault password file from the command line?

## SSH Authentication

We want to switch from password to ssh authentication. Create a playbook to install ssh keys.


In [13]:
!cat copy-key.yml


# Load variables from the following files
- hosts: course
  vars_files:
  - vaultfile.yml
  tasks:
  - name: Ensure /root/.ssh is present
    file: path=/root/.ssh owner=root group=root state=directory
  - name: Copy there my key
    copy: src=id_ansible.pub dest=/root/.ssh/authorized_keys mode=0600 owner=root group=root
    
    

In [14]:
!ansible-playbook copy-key.yml


PLAY [course] ******************************************************************

TASK [Gathering Facts] *********************************************************
fatal: [172.17.0.2]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.17.0.2 port 22: Connection refused\r\n", "unreachable": true}
ok: [172.17.0.4]
ok: [172.17.0.3]

TASK [Ensure /root/.ssh is present] ********************************************
ok: [172.17.0.4]
ok: [172.17.0.3]

TASK [Copy there my key] *******************************************************
changed: [172.17.0.4]
ok: [172.17.0.3]

PLAY RECAP *********************************************************************
172.17.0.2                 : ok=0    changed=0    unreachable=1    failed=0   
172.17.0.3                 : ok=3    changed=0    unreachable=0    failed=0   
172.17.0.4                 : ok=3    changed=1    unreachable=0    failed=0   



### Exercise

Comment out the ansible_password field  [in `inventory` here](/edit/notebooks/exercise-01/inventory)

  - guess the expected output without running ansible
  
  

In [16]:
# Running in debug mode we can see all ssh arguments injected via anisble. Discuss the DEBUG output       
!ansible -vvv -m ping all

Using /notebooks/exercise-01/ansible.cfg as config file
META: ran handlers
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/system/ping.py
<172.17.0.2> ESTABLISH SSH CONNECTION FOR USER: None
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/system/ping.py
<172.17.0.2> SSH: EXEC ssh -C -o ControlMaster=auto -o ControlPersist=60s -o StrictHostKeyChecking=no -o 'IdentityFile="id_ansible"' -o KbdInteractiveAuthentication=no -o PreferredAuthentications=gssapi-with-mic,gssapi-keyex,hostbased,publickey -o PasswordAuthentication=no -o ConnectTimeout=10 -o ControlPath=/root/.ansible/cp/e1be7488c1 172.17.0.2 '/bin/sh -c '"'"'echo ~ && sleep 0'"'"''
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/system/ping.py
<172.17.0.3> ESTABLISH SSH CONNECTION FOR USER: None
<172.17.0.4> ESTABLISH SSH CONNECTION FOR USER: None
<172.17.0.3> SSH: EXEC ssh -C -o ControlMaster=auto -o ControlPersist=60s -o StrictHostKeyChecking=no -o 'Ident

## Exercise

Run ansible in verbose mode to se all the injected ssh argument. If the output is too verbose, reduce it either with:
    - `--limit ipaddress` to contact only one node
    - host indexing/subscript eg: `all[0]` 

# Discuss vaults for:
- common secrets to be shared on a repo
- private secrets to reside on local PC